# Things you should know about Julia

Julia is high-level and easy to write (dynamically typed, for instance). At the same time it aims to be such that, when written carefully, it can be as fast as the fastest languages (C, Fortran). The secret ingredient? Just-in-time compilation (read up on it if you care).

Julia is specifically geared towards scientific and technical computing.

Julia is new. It doesn't have a huge number of libraries and a wide user base (yet).

Julia is not done. v0.6 came out in June and introduced several breaking changes.


# Basics of Julia, and IJulia notebooks, for the purpose of tensor networks.

In [3]:
# This is a comment  
println("My name is xxx") # replace xxx with your name and press Shift+Enter

My name is xxx


### numbers

In [4]:
a = 1
println(typeof(a))

Int64


In [6]:
b = 2.3
println(typeof(b))

Float64


In [8]:
c = a+b  # Beware of performance gotchas!
println(c)
println(typeof(c))

3.3
Float64


In [9]:
z2 = a + b*im  # complex numbers
z1 = complex(a, b)
z1 == z2

true

In [10]:
?complex

search: complex Complex Complex64 Complex32 Complex128 precompile __precompile__



```
complex(r, [i])
```

Convert real numbers or arrays to complex. `i` defaults to zero.

```
complex(T::Type)
```

Returns an appropriate type which can represent a value of type `T` as a complex number. Equivalent to `typeof(complex(zero(T)))`.

```jldoctest
julia> complex(Complex{Int})
Complex{Int64}

julia> complex(Int)
Complex{Int64}
```


### vectors

In [11]:
v = [1.0 2 0 0.2]  # row vector

1×4 Array{Float64,2}:
 1.0  2.0  0.0  0.2

In [12]:
w = [0.3; 0; 2; 1.2]  # column vector  

4-element Array{Float64,1}:
 0.3
 0.0
 2.0
 1.2

In [15]:
w = complex.(randn(4,1), randn(4,1))  # random column vector

4×1 Array{Complex{Float64},2}:
   0.637736-1.07667im
  -0.301651+0.47722im
    1.34004-1.71412im
 -0.0786585-1.12504im

In [23]:
w[2]  # access 2nd component of the vector w. Numbering starts from 1!

-0.30165094757918903 + 0.47722042912882806im

In [24]:
v*w  # scalar product as a 1x1 array

1×1 Array{Complex{Float64},2}:
 0.0187026-0.347234im

In [25]:
(v*w)[1]  # scalar product as a single number

0.018702635587533403 - 0.34723368405873706im

In [26]:
w'  # Conjugate transpose

1×4 Array{Complex{Float64},2}:
 0.637736+1.07667im  -0.301651-0.47722im  …  -0.0786585+1.12504im

In [31]:
sqrt((w'*w)[1]) == vecnorm(w)

true

### matrices

In [33]:
w*w'  # Outer product

4×4 Array{Complex{Float64},2}:
   1.56592+0.0im        -0.706181+0.0204369im  …    1.16113+0.802166im
 -0.706181-0.0204369im   0.318733+0.0im           -0.513163-0.376906im
   2.70013+0.349614im    -1.22224-0.122425im        1.82305+1.64242im 
   1.16113-0.802166im   -0.513163+0.376906im         1.2719+0.0im     

In [34]:
n = 3
M = rand(n, n)  # nxn random matrix

3×3 Array{Float64,2}:
 0.585952  0.665325  0.889817
 0.5905    0.20737   0.353809
 0.013864  0.777706  0.979527

In [35]:
M = (M + M')/2  # Let's make M Hermitian

3×3 Array{Float64,2}:
 0.585952  0.627912  0.45184 
 0.627912  0.20737   0.565757
 0.45184   0.565757  0.979527

### Eigenvalues and vectors

In [38]:
D, U = eig(M)

([-0.296521, 0.344115, 1.72526], [0.499306 -0.678199 -0.539203; -0.843673 -0.238888 -0.48078; 0.197255 0.694967 -0.691456])

In [40]:
@show D  # eigenvalues
@show U  # eigenvectors as rows

D = [-0.296521, 0.344115, 1.72526]
U = [0.499306 -0.678199 -0.539203; -0.843673 -0.238888 -0.48078; 0.197255 0.694967 -0.691456]


3×3 Array{Float64,2}:
  0.499306  -0.678199  -0.539203
 -0.843673  -0.238888  -0.48078 
  0.197255   0.694967  -0.691456

Is it true that $M = U \cdot D \cdot U^{\dagger}$? 

In [49]:
N = U*diagm(D)*U'  # let's build UDU'
M == N

false

In [50]:
M ≈ N

true

In [51]:
norm(M - N)

2.6304897551986876e-15

In [55]:
d1 = D[1]    # first eigenvalue of matrix M
u1 = U[:,1]  # first eigenvector of matrix M

3-element Array{Float64,1}:
  0.499306
 -0.843673
  0.197255

Is it true that $M\cdot\vec{u}_1 = d_1 \vec{u}_1$? 

In [56]:
M*u1 ≈ d1*u1

true

### Tensors

In [57]:
A_tensor = rand(2,2,3)

2×2×3 Array{Float64,3}:
[:, :, 1] =
 0.776811  0.570522
 0.946886  0.578174

[:, :, 2] =
 0.418961   0.184617
 0.0273454  0.210723

[:, :, 3] =
 0.436116  0.992994
 0.997976  0.985715

In [58]:
permutedims(A_tensor, (2,1,3))

2×2×3 Array{Float64,3}:
[:, :, 1] =
 0.776811  0.946886
 0.570522  0.578174

[:, :, 2] =
 0.418961  0.0273454
 0.184617  0.210723 

[:, :, 3] =
 0.436116  0.997976
 0.992994  0.985715

In [61]:
A_matrix = reshape(A_tensor, (4,3))

4×3 Array{Float64,2}:
 0.776811  0.418961   0.436116
 0.946886  0.0273454  0.997976
 0.570522  0.184617   0.992994
 0.578174  0.210723   0.985715

In [62]:
for i in 1:2
    for j in 1:2
        for k in 1:3
            println(A_tensor[i,j,k] == A_matrix[i + (j-1)*2, k])  # (i,j) -> i + (j-1)*2 is a bijection!
        end
    end
end

true
true
true
true
true
true
true
true
true
true
true
true


In [63]:
A_tensor == reshape(A_matrix, (2,2,3))

true

$C_{ij} = A_{ikm} B_{jkn} x_{mn}$

In [77]:
D = 8
d = 2
A=randn(D, d, D)
B=randn(D, d, D)
x=randn(D, D)

C = zeros(eltype(A), (D, D))
for i in 1:D
    for j in 1:D
        for k in 1:d
            for m in 1:D
                for n in 1:D
                    C[i,j] += A[i,k,m] * B[j,k,n] * x[m,n]
                end
            end
        end
    end
end
C

8×8 Array{Float64,2}:
   1.95282  -14.1955      7.05945  …    0.685361   -6.67212   13.307  
 -13.2314     0.561873    3.12435       0.441828   -1.25748    6.50263
  -2.70632    7.56017    -8.64605       0.884867    4.29495    4.2028 
  17.829     -2.63609    19.0705        6.37398     5.17074   12.8566 
  -8.19021   -8.33132     9.92002       0.37438    -9.24423   -3.98531
 -17.2215   -29.279      15.5893   …    0.398753   -7.64186    4.43791
  -1.41482  -10.406     -12.5167      -12.4666    -13.419      2.73614
   1.25496    2.46992    -6.01213      -4.12705     0.312403   5.40517

In [68]:
using TensorOperations

In [76]:
@tensor C2[i,j] := A[i,k,m] * (B[j,k,n] * x[m,n])
C2 ≈ C

true

### Functions and types

In [78]:
typeof(C)  # Parametric types!

Array{Float64,2}

In [79]:
MPSTensor{T} = Array{T,3}  # Define a new type, MPSTensor{T}, for every type T.

Array{T,3} where T

In [80]:
A = randn(3,3,3)
typeof(A) == MPSTensor{Float64}

true

In [82]:
# Define a composite type, that can hold several things as fields.
mutable struct UMPS{T}
    A::MPSTensor{T}
    # There could also be more fields, if for instance the MPS was of finite length:
    #lb::Vector{T}
    #rb::Vector{T}
    #length::Int32
end

In [83]:
M = UMPS(randn(4,2,4))  # Create a new instance of UMPS.
M.A

4×2×4 Array{Float64,3}:
[:, :, 1] =
 -0.624711   0.81219 
 -0.274045   1.58016 
  0.304923   0.091567
 -0.324495  -1.14701 

[:, :, 2] =
  1.61509   -1.15443 
 -0.781988  -0.913507
  1.42486    0.242666
 -0.721004  -0.821166

[:, :, 3] =
  0.783997   1.87251 
 -1.25609    0.136086
  0.969948  -0.383294
 -0.303481  -0.6856  

[:, :, 4] =
  1.34294   -0.063962
  0.245558   1.56128 
 -0.830528   1.00754 
 -0.709913   0.225044

In [89]:
function rand_UMPS(d::Int, D::Int)  # Specify that the arguments of rand_UMPS must be Ints.
    shp = (D, d, D)
    A = randn(shp)
    return UMPS{Float64}(A)
end

# Try modifying the above function so that it can take in a third argument,
# that would be the element type of the UMPS (above it's just always Float64).

rumps = rand_UMPS(2, 10)

UMPS{Float64}([-0.846168 -1.41074; -1.05757 -0.586876; … ; 0.312067 -1.64696; -0.378433 0.242243]

[0.2704 2.35381; 0.241059 0.710148; … ; 0.618658 1.01747; 0.249869 -0.0999388]

[-0.473256 0.976609; 0.00953401 -0.457308; … ; 0.570835 -2.50081; 0.0329324 0.351769]

[2.19616 -2.25986; -1.07877 0.705594; … ; 1.55647 -2.29203; 0.648942 0.911258]

[-0.810223 0.797905; 0.584915 2.12501; … ; -0.697768 -0.560918; -1.98353 0.403346]

[-0.793436 -0.0985046; 1.16135 -0.108438; … ; -0.478436 0.889807; 0.987962 -1.17988]

[-0.220849 -0.281321; -0.171815 -0.611377; … ; -0.416637 -1.69347; 0.453566 -0.435341]

[1.10113 0.55644; 1.16726 0.58958; … ; 0.0132259 1.94401; -0.977716 -0.469378]

[0.646013 1.1265; -0.79689 1.59958; … ; 0.988167 0.479808; 0.917322 -0.807249]

[-1.68409 0.223433; 2.59853 -0.112798; … ; -2.55285 0.549835; 0.538366 -0.0637573])

In [90]:
# Two different methods for the function bond_dim.
# Every time bond_dim is called, Julia looks at the type of the argument.
# If the argument is an MPSTensor, the first method is executed.
# If the argument is a UMPS, the second method is executed.
bond_dim(A::MPSTensor) = size(A, 1)
bond_dim(M::UMPS) = bond_dim(M.A)

bond_dim(rumps)

10

In [91]:
# Whenever a function modifies its arguments, the name should end in a "!".
# This is not syntax of the language, but just a convention.
# Please follow it though.
set_mps_tensor!{T}(M::UMPS{T}, A::MPSTensor{T}) = M.A = A

set_mps_tensor!(rumps, randn(3,2,3))
rumps

UMPS{Float64}([0.522837 1.00385; 0.805443 -1.41565; 0.0202699 0.842224]

[0.542974 -0.196122; 0.112067 1.24749; 0.396193 -2.39049]

[1.5308 -0.356433; -0.0432526 -1.26136; -1.49503 0.00949274])